In [17]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from edinburgh_challenge.constants import police_stations

In [18]:
source = "./data.xlsx"

In [27]:
data = pd.read_excel(source)
data["Time"] = (data["Day"]-1)*24 + data["Hour"]
data.columns = [x.lower() for x in data.columns]

In [96]:
class NaiveModel():
    def make_allocation(self, incidents, officers, current_time):
        # Sort incidents by priority
        incidents.sort(key=lambda inc: inc.priority)

        allocations = {}
        officers_allocated = []
        for inc in incidents:
            allocated = False
            # Sort stations by distance to the incident
            sorted_stations = sorted(inc.distances, key=inc.distances.get)

            for station in sorted_stations:
                # Check for available officer in the station
                available_officers = [off for off in officers[station] if (off.available and off not in officers_allocated) ]
                if available_officers:
                    # Allocate the first available officer
                    chosen_officer = available_officers[0]
                    allocations[inc.urn] = chosen_officer.name
                    officers_allocated.append(chosen_officer)
                    allocated = True
                    break
            
            if not allocated:
                # No officers available for this incident
                allocations[inc.urn] = None

        return allocations


In [97]:
from dataclasses import dataclass
from typing import List, Dict
import numpy as np
from abc import ABC, abstractmethod

class Model(ABC):
    @abstractmethod
    def make_allocation(self, incidents, officers, current_time):
        pass

@dataclass
class Incident:
    urn: str
    latitude: float
    longitude: float
    day:int
    hour:int
    global_time: float  # Time at which the incident was reported (24*day + hour)
    deployment_time: float  # Time taken to resolve the incident after reaching the location
    priority: str
    distances: Dict[str, float]  # Distances from each station
    resolved: bool = False
    resolving_officer: str = None
    response_time: float = None  # Global time when the response arrives at the scene
    resolution_time: float = None  # Global time when the incident is resolved

@dataclass
class Officer:
    name: str
    station: str
    available: bool = True
    return_time: float = 0.0
        
from typing import Dict, List

# Shift distribution structure
# Example: {'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5}, ...}
ShiftDistribution = Dict[str, Dict[str, int]]

In [230]:
from math import radians, cos, sin, asin, sqrt

class Simulation:
    SPEED_MPH = 30  # Speed in miles per hour

    def __init__(self, df: pd.DataFrame, station_coords: List[tuple], shift_distribution: ShiftDistribution, verbose:int=0):
        self.df = df.copy()
        self.station_coords = station_coords
        self.shift_distribution = shift_distribution
        self.current_time = 0
        self.officers = { key: [] for key in shift_distribution["Early"].keys() } # Station_1, Station_2 and Station_3
        self.verbose = verbose

        # This resolved incidents is made to keep a track of when an incident
        # is allocated to an officer. This is to conduct the evaluating model check
        self.resolved_incidents = [] # This list keeps a track and order of the resolved incidents as they are allocated
    
        
        # Making the df compatible with data structures
        df.columns = [x.lower() for x in df.columns]
        

    def print_dashboard(self, allocations):
        print(f"\n--- Day {self.current_time // 24 + 1}, Hour {self.current_time % 24} ---")
        pending_cases = [inc for inc in self.cumulative_incidents if not inc.resolved]
        print(f"Pending Cases: {len(pending_cases)}")
        print("Pending Incident URNs:", [inc.urn for inc in pending_cases])

        inv_allocations = {v: k for k, v in allocations.items()}
        
        for station, officers in self.officers.items():
            print(f"\n{station}:")
            for officer in officers:
                # Find the incident the officer is currently assigned to
                if officer.name in inv_allocations.keys():
                    status = inv_allocations[officer.name]
                else: 
                    status = "Busy"
                print(f"  - {officer.name}: {status}")
        
    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees)
        """
        # Convert decimal degrees to radians 
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Haversine formula 
        dlat = lat2 - lat1 
        dlon = lon2 - lon1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 3956  # Radius of Earth in miles. Use 6371 for kilometers
        return c * r

    def generate_incidents_for_hour(self, global_time):
        hour_incidents = self.df[(self.df['time'] == global_time)]
        incidents = []
        for _, row in hour_incidents.iterrows():
            deployment_time = row.pop("deployment time (hrs)")
            row.pop("time")  # Not needed as we use global_time
            distances = {f'Station_{i+1}': self.calculate_distance(row['latitude'], row['longitude'], lat, lon) 
                         for i, (lat, lon) in enumerate(self.station_coords)}
            incidents.append(Incident(**row, 
                                      deployment_time=deployment_time, 
                                      distances=distances,
                                      global_time=global_time))
        return incidents


    def update_officers_for_shift(self, shift):
        for station, num_officers in self.shift_distribution[shift].items():
            # Update the number of officers for each station
            self.officers[station] = [Officer(f"Officer_{station}_{shift}_{i}", station=station) for i in range(num_officers)]

    def process_allocations(self, allocations):
        if self.verbose > 0:
            print(f"{allocations=}")
        for urn, officer_id in allocations.items():
            incident = next((inc for inc in self.cumulative_incidents if inc.urn == urn), None)
            #print("incident:", incident)
            if officer_id is None:
                if incident:
                    incident.resolved = False
                continue

            officer = next((off for station_officers in self.officers.values() for off in station_officers if off.name == officer_id), None)
            if incident and officer:
                travel_time = incident.distances[officer.station] / self.SPEED_MPH
                officer.available = False  # Mark officer as busy
                officer.return_time = self.current_time + travel_time + incident.deployment_time
                
                incident.resolved = True  # Mark incident as resolved
                incident.resolving_officer = officer.name  # Assign officer to incident
                incident.response_time = self.current_time + travel_time # Global time when the response reached
                incident.resolution_time = officer.return_time # Global Time when the incident was resolved
                self.resolved_incidents.append(incident)

                
    def update_officer_availability(self):
        for station, officers in self.officers.items():
            for officer in officers:
                if not officer.available and self.current_time >= officer.return_time:
                    officer.available = True


    def run(self, model):
        self.cumulative_incidents = []  # Global list to track all incidents

        while self.current_time < 24 * 6:  # For a week-long simulation
            
            day = self.current_time // 24 + 1
            hour = self.current_time % 24
            
            # Update officer availability at the start of each timestep
            self.update_officer_availability()

            # Update officers for shift change
            if hour in [0, 8, 16]:
                shift = 'Early' if hour == 0 else 'Day' if hour == 8 else 'Night'
                #print(f"{shift=}")
                self.update_officers_for_shift(shift)
            
            total_officers = len(self.officers["Station_1"]) + len(self.officers["Station_2"]) + len(self.officers["Station_3"]) 
            #print(f"{total_officers=}")
                
            # Generate and add new incidents
            new_incidents = self.generate_incidents_for_hour(self.current_time)
            self.cumulative_incidents.extend(new_incidents)
            
            # Filter to get only pending incidents
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{len(pending_incidents)=}")
            #print(f"{pending_incidents=}")
            # Call the model to allocate incidents
            #print(f"{self.officers=}")
            allocations = model.make_allocation(pending_incidents, self.officers, self.current_time)
            
            # Process allocations and update the state
            self.process_allocations(allocations)

            if self.verbose > 2:
                self.print_dashboard(allocations)
            
            
            # Pending cases after allocation
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{pending_incidents=}")
            
            self.current_time += 1  # Move to the next hour
            if self.verbose > 3:
                input("Press Enter to continue to the next hour...\n")
            #print()
            #print([inc.urn for inc in self.cumulative_incidents])
            #print("*"*50)

    # Checks and Analysis
    def analyze_simulation_results(self):

        simulation = self
        
        # Initialize counters and accumulators
        incident_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        resolved_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        within_threshold_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        response_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        resolution_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        deployment_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        thresholds = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}
        total_officer_hours = {officer.name: 0 for station in simulation.officers.values() for officer in station}
        unresolved_incidents = 0
    
        # Analyze each incident
        for incident in simulation.cumulative_incidents:
            priority = incident.priority
            incident_counts[priority] += 1
    
            if incident.resolved:
                resolved_counts[priority] += 1
                
                response_time = incident.response_time
                resolution_time = incident.resolution_time
                
                response_times[priority].append(response_time)
                resolution_times[priority].append(resolution_time)
                
                deployment_time = incident.deployment_time
                deployment_times[priority].append(deployment_time)
                
                # Calculate the time from incident report to response arrival
                time_to_response = incident.response_time - incident.global_time
    
    
                # Check if the response was within the threshold
                if time_to_response <= thresholds[priority]:
                    within_threshold_counts[priority] += 1
            else:
                unresolved_incidents += 1
    
        # Calculate percentages and mean times
        completion_percentages = {p: (resolved_counts[p] / incident_counts[p]) * 100 if incident_counts[p] > 0 else 0 for p in incident_counts}
        mean_response_times = {p: np.mean(response_times[p]) if response_times[p] else 0 for p in response_times}
        mean_deployment_times = {p: np.mean(deployment_times[p]) if deployment_times[p] else 0 for p in deployment_times}
        threshold_compliance = {p: (within_threshold_counts[p] / resolved_counts[p]) * 100 if resolved_counts[p] > 0 else 0 for p in incident_counts}
    
        # Calculate officer utilization
        for station in simulation.officers.values():
            for officer in station:
                if not officer.available:
                    total_officer_hours[officer.name] += (simulation.current_time - officer.return_time)
        officer_utilization = sum(total_officer_hours.values()) / (len(total_officer_hours) * simulation.current_time) * 100
    
        unresolved_incident_percentage = (unresolved_incidents / len(simulation.cumulative_incidents)) * 100 if simulation.cumulative_incidents else 0
    
        return {
            "Completion Percentages": completion_percentages,
            "Mean Response Times": mean_response_times,
            "Mean Deployment Times": mean_deployment_times,
            "Threshold Compliance": threshold_compliance,
            "Officer Utilization": officer_utilization,
            "Unresolved Incident Percentage": unresolved_incident_percentage
        }

    def check_simulation(self):
        simulation = self
        # Initialize officer assignments based on all shifts
        officer_assignments = {}
        for shift, stations in shift_distribution.items():
            for station, num_officers in stations.items():
                for i in range(num_officers):
                    officer_name = f"Officer_{station}_{shift}_{i}"
                    officer_assignments[officer_name] = []
        
        incident_response = {'Immediate': {'total': 0, 'within_time': 0}, 
                             'Prompt': {'total': 0, 'within_time': 0}, 
                             'Standard': {'total': 0, 'within_time': 0}}
        
        time_travel_occurred = False
    
        for incident in simulation.resolved_incidents:
            if incident.resolved:
                # Check officer assignments and time traveling
                if incident.resolving_officer:
                    officer_assignments[incident.resolving_officer].append(incident.resolution_time)
                    if len(officer_assignments[incident.resolving_officer]) > 1:
                        if officer_assignments[incident.resolving_officer][-2] > incident.resolution_time:
                            #print(f"{officer_assignments[incident.resolving_officer]=}")
                            #print(f"{incident.resolving_officer=}")
                            time_travel_occurred = True
    
                # Count incidents and check response time
                incident_response[incident.priority]['total'] += 1
                target_time = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}[incident.priority]
                if incident.response_time - incident.global_time <= target_time:
                    incident_response[incident.priority]['within_time'] += 1
    
        # Calculate percentages
        for priority in incident_response:
            total = incident_response[priority]['total']
            if total > 0:
                incident_response[priority]['percentage'] = (incident_response[priority]['within_time'] / total) * 100
            else:
                incident_response[priority]['percentage'] = 0
    
        return officer_assignments, incident_response, time_travel_occurred

In [231]:
shift_distribution = {
    'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]
num_officers_per_station = [4, 5, 6]
simulation = Simulation(data, ps_coords, shift_distribution, 
                        verbose=1)

In [232]:
naive_model = NaiveModel()

In [233]:
simulation.run(naive_model)

allocations={'PS-20220709-0475': 'Officer_Station_1_Early_0', 'PS-20220711-0540': 'Officer_Station_1_Early_1', 'PS-20220711-0568': 'Officer_Station_1_Early_2', 'PS-20220706-0009': 'Officer_Station_3_Early_0', 'PS-20220706-0021': 'Officer_Station_2_Early_0', 'PS-20220706-0028': 'Officer_Station_1_Early_3', 'PS-20220706-0035': 'Officer_Station_2_Early_1', 'PS-20220706-0043': 'Officer_Station_3_Early_1', 'PS-20220706-0054': 'Officer_Station_2_Early_2', 'PS-20220706-0091': 'Officer_Station_1_Early_4', 'PS-20220706-0092': 'Officer_Station_3_Early_2', 'PS-20220709-0465': 'Officer_Station_2_Early_3', 'PS-20220709-0467': 'Officer_Station_2_Early_4', 'PS-20220709-0493': 'Officer_Station_3_Early_3', 'PS-20220709-0496': 'Officer_Station_3_Early_4', 'PS-20220709-0502': None, 'PS-20220709-0527': None, 'PS-20220709-0540': None, 'PS-20220711-0538': None, 'PS-20220711-0546': None, 'PS-20220711-0547': None, 'PS-20220711-0563': None, 'PS-20220711-0588': None, 'PS-20220711-0589': None, 'PS-20220711-0592'

In [234]:
officer_assignments, incident_response, time_travel_occurred = simulation.check_simulation()
if time_travel_occurred:
    print("ERROR: Time Travel Occured")

In [235]:
analysis = simulation.analyze_simulation_results()
analysis

{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 97.26651480637814},
 'Mean Response Times': {'Immediate': 76.69943973316968,
  'Prompt': 72.79034842356631,
  'Standard': 81.4700081932075},
 'Mean Deployment Times': {'Immediate': 1.5379699248120302,
  'Prompt': 1.5070445344129555,
  'Standard': 1.4734192037470728},
 'Threshold Compliance': {'Immediate': 99.62406015037594,
  'Prompt': 94.8178137651822,
  'Standard': 28.805620608899297},
 'Officer Utilization': -0.11015434113684058,
 'Unresolved Incident Percentage': 0.6185567010309279}

### Simulation Checks and Analysis

Currently, the simulation check says that there is time travelling going on. However, this is a false alarm. The simulation thinks that as we read the events in the order of cumulative list(these are ordered as they are received. Basically, this order should be the same as the order of the dataset). However, beaucase incidents are prioritised in accordance to their priority, there is small chance that an incident with a lower urn but higher priority is done first. The simulation thinks that this is proof of time travelling. However, this isn't how we should be checking for time travelling.

Update: Added a list called resolved_incident to simulation that keeps a track of when something was assigned to a person (essentailly). Hence, it becomes easier to check for time travelling on this list instead of cumulative list. The logic of the check_simulation function was not changed.

### EnhancedModel testing

In [243]:
class EnhancedModel(Model):
    def make_allocation(self, incidents, officers, current_time):
        # Adjusting the priority mechanism to balance between priority, waiting time, and travel time
        incidents.sort(key=lambda inc: (inc.priority, current_time - inc.global_time, min(inc.distances.values())))

        allocations = {}
        allocated_officers = set()  # Set to keep track of officers already allocated

        for inc in incidents:
            # Find the nearest station with available officers
            nearest_stations = sorted(inc.distances, key=inc.distances.get)

            for station in nearest_stations:
                # Filter out officers who are already allocated
                available_officers = [off for off in officers[station] if off.available and off.name not in allocated_officers]

                if available_officers:
                    # Allocate the first available officer
                    chosen_officer = available_officers[0]
                    allocations[inc.urn] = chosen_officer.name
                    allocated_officers.add(chosen_officer.name)  # Mark officer as allocated
                    chosen_officer.available = False  # Mark officer as busy
                    # Assuming return_time is calculated elsewhere
                    break
            else:
                # No officers available for this incident
                allocations[inc.urn] = None

        return allocations


In [244]:
shift_distribution = {
    'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]
num_officers_per_station = [4, 5, 6]
simulation = Simulation(data, ps_coords, shift_distribution, 
                        verbose=1)

In [245]:
enhanced_model = EnhancedModel()
simulation.run(enhanced_model)

allocations={'PS-20220711-0540': 'Officer_Station_1_Early_0', 'PS-20220709-0475': 'Officer_Station_1_Early_1', 'PS-20220711-0568': 'Officer_Station_1_Early_2', 'PS-20220711-0592': 'Officer_Station_1_Early_3', 'PS-20220709-0502': 'Officer_Station_3_Early_0', 'PS-20220709-0493': 'Officer_Station_1_Early_4', 'PS-20220706-0091': 'Officer_Station_2_Early_0', 'PS-20220709-0496': 'Officer_Station_2_Early_1', 'PS-20220709-0465': 'Officer_Station_2_Early_2', 'PS-20220706-0054': 'Officer_Station_2_Early_3', 'PS-20220709-0540': 'Officer_Station_2_Early_4', 'PS-20220711-0563': 'Officer_Station_3_Early_1', 'PS-20220711-0589': 'Officer_Station_3_Early_2', 'PS-20220706-0092': 'Officer_Station_3_Early_3', 'PS-20220711-0547': 'Officer_Station_3_Early_4', 'PS-20220706-0028': None, 'PS-20220709-0467': None, 'PS-20220706-0043': None, 'PS-20220709-0527': None, 'PS-20220711-0538': None, 'PS-20220711-0588': None, 'PS-20220711-0606': None, 'PS-20220711-0645': None, 'PS-20220706-0021': None, 'PS-20220706-0035'

In [246]:
officer_assignments, incident_response, time_travel_occurred = simulation.check_simulation()
if time_travel_occurred:
    print("ERROR: Time Travel Occured")

In [247]:
analysis = simulation.analyze_simulation_results()
analysis

{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 95.8997722095672},
 'Mean Response Times': {'Immediate': 76.6963228218582,
  'Prompt': 72.79588645341676,
  'Standard': 81.28011322019752},
 'Mean Deployment Times': {'Immediate': 1.5379699248120302,
  'Prompt': 1.5070445344129555,
  'Standard': 1.4794536817102137},
 'Threshold Compliance': {'Immediate': 100.0,
  'Prompt': 91.98380566801619,
  'Standard': 62.70783847980997},
 'Officer Utilization': -0.12816666972406526,
 'Unresolved Incident Percentage': 0.9278350515463918}

In [ ]:
{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 97.26651480637814},
 'Mean Response Times': {'Immediate': 76.69943973316968,
  'Prompt': 72.79034842356631,
  'Standard': 81.4700081932075},
 'Mean Deployment Times': {'Immediate': 1.5379699248120302,
  'Prompt': 1.5070445344129555,
  'Standard': 1.4734192037470728},
 'Threshold Compliance': {'Immediate': 99.62406015037594,
  'Prompt': 94.8178137651822,
  'Standard': 28.805620608899297},
 'Officer Utilization': -0.11015434113684058,
 'Unresolved Incident Percentage': 0.6185567010309279}